# Wrangle OpenStreetMap Data with SQL

By Trevor Cook

<h3>Project Summary</h3>

This project goes through the data wrangling process by extracting a metropolitan area on OpenStreetMap, converting it into xml format, uploading the data into a database, and performing queries on the database with SQL. Before exporting the data into xml files, an audit is performed to analyze whether there are any problems or inconsistencies with the data. These problems, such as inconsistent streen names, are then updated before exporting the data into xml files.

Map area: Halifax, Nova Scotia

The map position that I chose for this project can be found at the following link: <br>https://www.openstreetmap.org/relation/1339656#map=10/44.7857/-63.2304

The reason for choosing Halifax, NS as the OpenStreetMap area for my project is because I spent 4 years in this city attending Dalhousie University. Halifax is and always will be one of my favorite cities.

Map size: <br>
halifax_canada.osm: 80.4 MB

<h3>Table of Contents</h3>

1) Data Audit <br>
2) Problems Encountered <br>
3) Converting .csv files into Database <br>
4) Queries <br>
5) Conclusion

# Data Audit

In [1]:
# The schema that will be used for when converting the osm data into csv format

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string', 'required': True}
            }
        }
    }
}


<h3>Street Names</h3>

In [3]:
import xml.etree.cElementTree as ET
import re
from collections import defaultdict
import pprint

# Makes a list of the street abbreviations and stores them in a set
# Audits the street names so that you can view any potential problems

osm_file = open("halifax_canada.osm", "r")

# This regular expression looks at the last word in the string as this is where I expect to see the street abbreviation show up.
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

# Here is a list of several common street abbreviations that I expect to see in the data. 
expected = ["Street", "Avenue", "Boulevard", "Drive", "Place", "Road", "Court", "Lane"]

def audit_street_type(street_types, street_name):
    '''This audit pulls out unexpected street abbreviations which are not in this list above'''
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)
        
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))

audit()


{'Blvd': set(['Topsail Blvd']),
 'Circle': set(['South Ridge Circle']),
 'Close': set(['Lanshaw Close', 'Loppie Close', 'Roxham Close']),
 'Crescent': set(['Digby Crescent', 'Plateau Crescent']),
 'Dr': set(['Novalea Dr']),
 'Grove': set(['Promise Grove']),
 'Highway': set(['Bedford Highway']),
 'Hill': set(['Ioney Hill']),
 'NS': set(['MUMFORD ROAD, HALIFAX NS']),
 'Quinpool': set(['6487 Quinpool']),
 'Rd': set(['6324 Quinpool Rd', 'Kempt Rd', 'Quinpool Rd']),
 'Rd.': set(['6371 Quinpool Rd.']),
 'Row': set(['Dresden Row']),
 'Run': set(['Beech Tree Run']),
 'St': set(['Gesner St', 'May St', 'Preston St']),
 'St,': set(['Provo Wallis St,']),
 'St.': set(['George St.']),
 'Terrace': set(['Cannon Terrace', 'Milo Terrace']),
 'Walk': set(['Lemon Walk']),
 'Way': set(['Peakview Way', 'Whispering Way']),
 'West': set(['Portland Estates Boulevard West', 'Sullivan Street West']),
 'ave': set(['millbrook ave'])}


Based on this audit, we can see that there are several instances where the street abbreviations were not found in the expected list of names.

The first type represents uncommon street abbreviations, such as Circle or Crescent. There is nothing wrong with these street types, they are simply more rare names that have not been included in the expected list.

The second type is shortened street names, for example Dr or St. This data will have to be cleaned up by changing their names to Drive and Street, respectively. 

The data audit also shows an instance when the contributor included 'HALIFAX NS' in the street address.

<h3>Postal Codes</h3>

In [4]:
# Makes a list of the postal codes and stores them in a set
# Audits the postal codes so that you can view any potential problems

osm_file = open("halifax_canada.osm", "r")

# This regular expression looks at the first block of text in the string
postal_type_re = re.compile(r'\S+\.?', re.IGNORECASE)
postal_types = defaultdict(set)

# This is a list of the first 3 digits of postal codes found in Halifax
expected = ["B2W", "B2X", "B2Y", "B2Z", "B3B", "B3H", "B3J", "B3K", "B3L", "B3M", "B3N", "B3S", "B3T", "B4A", "B4C", "B4E"]

def audit_postal_type(postal_types, postal_name):
    '''This audit pulls out unexpected postal codes which are not in the list expected'''
    m = postal_type_re.search(postal_name)
    if m:
        postal_type = m.group()
        if postal_type not in expected:
            postal_types[postal_type].add(postal_name)
            
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)
        
def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit():
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            # Returns all subtags with name 'tag'
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_type(postal_types, tag.attrib['v'])
    pprint.pprint(dict(postal_types))

audit()

{'B3L1B1': set(['B3L1B1']),
 'B3L1J6': set(['B3L1J6']),
 'B3S0E1': set(['B3S0E1']),
 'B4E3B5': set(['B4E3B5']),
 'NS': set(['NS B3J 1P2', 'NS B3L 1A6']),
 'b2v0a2': set(['b2v0a2'])}


The audit for postal codes is similar to that done for the street types as it highlights any potential problems with the data. As we can see from the audit, there are a few opportunities for cleaning this data. The first problem is that not all of the postal codes are written in capital letters. There are also many cases where there is whitespace missing between each 3 character block. Finally, some postal codes begin with 'NS', followed by the postal code.

After cleaning the data, each postal code will be written in the same format. They will be written in uppercase letters, and will contain a space following the first 3-character block of the postal code.

# Problems Encountered

<h3>Street Names</h3>

In [16]:
# Takes the problem street name and updates it to the correct name

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "halifax_canada.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# Street names that need to be updated based on the audit.
mapping = { "St": "Street",
            "St.": "Street",
            "St,": "Street",
            "Ave": "Avenue",
            "ave": "Avenue",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            }


def audit_street_type(street_types, street_name):
    '''Adds the street type to a dictionary if it is not found in the 'expected' list'''
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            

def is_street_name(elem):
    '''Verifies whether the arrribute's key corresponds to a street type'''
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    '''Changes the incorrect street abbreviation to the correct abbreviation, as shown in the mapping dict.'''
    name_split = name.split()
    for key in mapping.keys():
        if key in name_split:
            name = ' '.join(name_split)
            name = name.replace(key, mapping[key])
    return name


def test():
    st_types = audit(OSMFILE)

    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name



test()

{'101-01': set(['Service Road HFX 101-01']),
 '104': set(['1546 Barrington Street #104']),
 '107': set(['Highway 107']),
 '12': set(['1475 Lower Water Street, Suite 12']),
 '2': set(['Highway 2']),
 '3Z2': set(['1475 Lower Water St, Halifax, NS B3J 3Z2']),
 '7': set(['Highway 7']),
 'Arm': set(['Belmont On The Arm']),
 'Ave': set(['Garland Ave']),
 'Barrington': set(['Barrington']),
 'Blvd': set(['Topsail Blvd']),
 'Chebucto': set(['Chebucto']),
 'Circle': set(['Blockade Circle',
                'Chebucto Circle',
                'Edzell Castle Circle',
                'Fieldwood Circle',
                'Haida Circle',
                'Lochiel Circle',
                'South Ridge Circle',
                'Wentworth Circle']),
 'Close': set(['Banbury Close',
               'Baroness Close',
               'Berkshire Close',
               'Brockhurst Close',
               'Cairnwell Close',
               'Candytuft Close',
               'Canterbury Close',
               'Caxton Cl

<h3>Postal Codes</h3>

In [18]:
OSMFILE = "halifax_canada.osm"
postal_type_re = re.compile(r'\b\S+\.?', re.IGNORECASE)

# This is a list of the first 3 digits of postal codes found in Halifax
expected_postal = ["B2W", "B2X", "B2Y", "B2Z", "B3B", "B3H", "B3J", "B3K", "B3L", "B3M", "B3N", "B3S", "B3T", "B4A", "B4C", "B4E"]

def audit_postal_type(postal_types, postal_name):
    '''Adds the postal code to a dictionary if it is not found in the 'expected' list'''
    m = postal_type_re.search(postal_name)
    if m:
        postal_type = m.group()
        if postal_type not in expected_postal:
            postal_types[postal_type].add(postal_name)
            
def is_postal_code(elem):
    '''Verifies whether the attribute's key corresponds to a postal code'''
    return (elem.attrib['k'] == "addr:postcode")


def audit_postal(osmfile):
    osm_file = open(osmfile, "r")
    postal_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_type(postal_types, tag.attrib['v'])
    osm_file.close()
    return postal_types


def update_postal(name):
    '''Changes the postal codes to a consistent format'''
    if name[0] == "b":
        name = name.upper() # Capitalize lower case postal codes
    if name[0] == "N":
        name = name.split(" ")
        first, second = name[1], name[2]
        seq = (first, second)
        name = " ".join(seq) # Remove 'NS' from beginning of string
    if "-" in name:
        name = name.replace("-", " ") # Changes '-' to a space
    else:
        if len(name) == 6:
            # If the length of the string is 6, there is no space in between each 3-character block
            # This code splits the string in half and adds a space between them
            # Reference: http://stackoverflow.com/questions/4789601/split-a-string-into-2-in-python
            first, second = name[:len(name)/2], name[len(name)/2:]
            seq = (first, second)
            s = " "
            name = s.join(seq)      
    return name


def test():
    postal_types = audit_postal(OSMFILE)

    pprint.pprint(dict(postal_types))

    for postal_type, ways in postal_types.iteritems():
        for name in ways:
            better_name = update_postal(name)
            print name, "=>", better_name



test()

{'B3A': set(['B3A 1M6']),
 'B3J1K1': set(['B3J1K1']),
 'B3J1S5': set(['B3J1S5']),
 'B3J1T2': set(['B3J1T2']),
 'B3J1T3': set(['B3J1T3']),
 'B3J1Z6': set(['B3J1Z6']),
 'B3J3J4': set(['B3J3J4']),
 'B3J3M7': set(['B3J3M7']),
 'B3J3N4': set(['B3J3N4']),
 'B3K1Y5': set(['B3K1Y5']),
 'B3K2R4': set(['B3K2R4']),
 'B3K4X5': set(['B3K4X5']),
 'B3L1A7': set(['B3L1A7']),
 'B3L1B1': set(['B3L1B1']),
 'B3L1B2': set(['B3L1B2']),
 'B3L1B6': set(['B3L1B6']),
 'B3L1J6': set(['B3L1J6']),
 'B3N2E5': set(['B3N2E5']),
 'B3S0E1': set(['B3S0E1']),
 'B3S1M8': set(['B3S1M8']),
 'B3S1R1': set(['B3S1R1']),
 'B4A-2R8': set(['B4A-2R8']),
 'B4A4H5': set(['B4A4H5']),
 'B4E3B5': set(['B4E3B5']),
 'Halifax': set(['Halifax']),
 'NS': set(['NS B3J 1P2', 'NS B3L 1A6', 'NS B4E 1R7']),
 'b2v0a2': set(['b2v0a2']),
 'b3J': set(['b3J 2A4']),
 'b3k': set(['b3k 6r8']),
 'b3k3b4': set(['b3k3b4'])}
B3J3N4 => B3J 3N4
b3k 6r8 => B3K 6R8
B3J1S5 => B3J 1S5
B3K2R4 => B3K 2R4
B4A-2R8 => B4A 2R8
Halifax => Halifax
B3N2E5 => B3N 2E5
B4A4H

In [22]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET

import cerberus

OSM_PATH = "/Users/tcook/Desktop/openStreetMap/halifax_canada.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


# ================================================== #
#               Update Name Functions                #
# ================================================== #

OSMFILE = "halifax_canada.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
postal_type_re = re.compile(r'\b\S+\.?', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# This is a list of the first 3 digits of postal codes found in Halifax
expected_postal = ["B2W", "B2X", "B2Y", "B2Z", "B3B", "B3H", "B3J", "B3K", "B3L", "B3M", "B3N", "B3S", "B3T", "B4A", "B4C", "B4E"]

# Street names that need to be updated based on the audit.
mapping = { "St": "Street",
            "St.": "Street",
            "St,": "Street",
            "Ave": "Avenue",
            "ave": "Avenue",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            }

def audit_street_type(street_types, street_name):
    '''Adds the street type to a dictionary if it is not found in the 'expected' list'''
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            

def is_street_name(elem):
    '''Verifies whether the arrribute's key corresponds to a street type'''
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_street(name, mapping):
    name_split = name.split()
    for key in mapping.keys():
        if key in name_split:
            name = ' '.join(name_split)
            name = name.replace(key, mapping[key])
    return name


# ===========  Postal Codes ===========  #

def audit_postal_type(postal_types, postal_name):
    '''Adds the postal code to a dictionary if it is not found in the 'expected' list'''
    m = postal_type_re.search(postal_name)
    if m:
        postal_type = m.group()
        if postal_type not in expected_postal:
            postal_types[postal_type].add(postal_name)
            
def is_postal_code(elem):
    '''Verifies whether the attribute's key corresponds to a postal code'''
    return (elem.attrib['k'] == "addr:postcode")


def audit_postal(osmfile):
    osm_file = open(osmfile, "r")
    postal_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_type(postal_types, tag.attrib['v'])
    osm_file.close()
    return postal_types


def update_postal(name):
    '''Changes the postal codes to a consistent format'''
    if name[0] == "b":
        name = name.upper() # Capitalize lower case postal codes
    if name[0] == "N":
        name = name.split(" ")
        first, second = name[1], name[2]
        seq = (first, second)
        name = " ".join(seq) # Remove 'NS' from beginning of string
    if "-" in name:
        name = name.replace("-", " ") # Changes '-' to a space
    else:
        if len(name) == 6:
            # If the length of the string is 6, there is no space in between each 3-character block
            # This code splits the string in half and adds a space between them
            # Reference: http://stackoverflow.com/questions/4789601/split-a-string-into-2-in-python
            first, second = name[:len(name)/2], name[len(name)/2:]
            seq = (first, second)
            s = " "
            name = s.join(seq)      
    return name



# ================================================== #
#               Shape Element Function               #
# ================================================== #


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    # Reference: https://discussions.udacity.com/t/help-cleaning-data/169833/9
    # Traps the 'user' and 'uid's in NODE_FIELDS which have no values
    if element.tag == 'node':
        for node in NODE_FIELDS:
            try:
                node_attribs[node] = element.attrib[node]
            except:
                node_attribs[node] = "00000"
                pass
            
        
        for child in element:            
            if child.tag == 'tag':
                if is_street_name(child):
                    # Update street names
                    child.attrib['v'] = update_street(child.attrib['v'], mapping)
                elif is_postal_code(child):
                    #Update postal codes
                    child.attrib['v'] = update_postal(child.attrib['v'])

            
            node_tags_dict = {}
            node_tags_dict['id'] = element.attrib['id']
            node_tags_dict['value'] = child.attrib['v']
            m = PROBLEMCHARS.match(child.attrib['k'])
                        
            
            if m:
                continue
            else:
                if ':' in child.attrib['k']:
                    key_split = child.attrib['k'].split(":", 1)
                    node_tags_dict['type'] = key_split[0]
                    node_tags_dict['key'] = key_split[1]
                    tags.append(node_tags_dict)
                else:
                    node_tags_dict['type'] = 'regular'
                    node_tags_dict['key'] = child.attrib['k']
                    tags.append(node_tags_dict)
                            
        
        return {'node': node_attribs, 'node_tags': tags}

    
    elif element.tag == 'way':              
        
        for child in element:
            if child.tag == 'tag':
                if is_street_name(child):
                    # Update street names
                    child.attrib['v'] = update_street(child.attrib['v'], mapping)
                elif is_postal_code(child):
                    #Update postal codes
                    child.attrib['v'] = update_postal(child.attrib['v'])                     
        
        for field in WAY_FIELDS:
            way_attribs[field] = element.attrib[field]
            
        position = 0
        for child in element:
            if child.tag == 'tag':
                way_tags_dict = {}
                way_tags_dict['id'] = element.attrib['id']
                way_tags_dict['value'] = child.attrib['v']
                m = PROBLEMCHARS.match(child.attrib['k'])
                if m:
                    continue
                else:
                    if ':' in child.attrib['k']:
                        key_split = child.attrib['k'].split(":", 1)
                        way_tags_dict['type'] = key_split[0]
                        way_tags_dict['key'] = key_split[1]
                        tags.append(way_tags_dict)
                    else:
                        way_tags_dict['type'] = 'regular'
                        way_tags_dict['key'] = child.attrib['k']
                        tags.append(way_tags_dict)
        
            elif child.tag == 'nd':
                way_nodes_dict = {}
                way_nodes_dict['id'] = element.attrib['id']
                way_nodes_dict['node_id'] = child.attrib['ref']
                way_nodes_dict['position'] = position
                position += 1
                way_nodes.append(way_nodes_dict)
                    
                        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
#     Note: Validation is ~ 10X slower. For the project consider using a small
#     sample of the map when validating.
    process_map(OSM_PATH, validate=True)


# Converting .csv files into Database

In [23]:
import sqlite3
import csv
from pprint import pprint

# Reference: http://stackoverflow.com/questions/2887878/importing-a-csv-file-into-a-sqlite3-database-table-using-python

sqlite_file = 'halifaxdb.db'
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

# Create the tables
cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')
cur.execute('''
    CREATE TABLE ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')
cur.execute('''
    CREATE TABLE ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)
''')
cur.execute('''
    CREATE TABLE nodes(id INTEGER, lat DOUBLE PRECISION, lon DOUBLE PRECISION, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TIMESTAMP)
''')
cur.execute('''
    CREATE TABLE ways(id INTEGER, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TIMESTAMP)
''')

conn.commit()

# Read in the .csv files as a dict.
with open('nodes_tags.csv', 'rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]

with open('ways_tags.csv', 'rb') as fin2:
    dr2 = csv.DictReader(fin2)
    to_db2 = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr2]

with open('ways_nodes.csv', 'rb') as fin3:
    dr3 = csv.DictReader(fin3)
    to_db3 = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in dr3]

with open('nodes.csv', 'rb') as fin4:
    dr4 = csv.DictReader(fin4)
    to_db4 = [(i['id'].decode("utf-8"), i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr4]

with open('ways.csv', 'rb') as fin5:
    dr5 = csv.DictReader(fin5)
    to_db5 = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr5]
    
# Insert formatted data
cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db2)
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db3)
cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db4)
cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db5)



# Queries

In [24]:
# Number of nodes

cur.execute('''
SELECT count(*) FROM nodes
''')
all_rows = cur.fetchall()
print "1) Number of nodes"
pprint(all_rows)

1) Number of nodes
[(379161,)]


In [25]:
# Number of ways

cur.execute('''
SELECT count(*) FROM ways
''')
all_rows = cur.fetchall()
print "2) Number of ways"
pprint(all_rows)

2) Number of ways
[(33193,)]


In [26]:
# Number of unique users

cur.execute('''
SELECT count(DISTINCT(e.uid))
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) as e
''')
all_rows = cur.fetchall()
print "3) Number of unique users"
pprint(all_rows)

3) Number of unique users
[(355,)]


In [27]:
# Top 10 contributors to the Halifax OpenStreetMap

cur.execute('''
SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10
''')
all_rows = cur.fetchall()
print "4) Top 10 contributors"
pprint(all_rows)

4) Top 10 contributors
[(u'macgregor', 190976),
 (u'timdine', 136514),
 (u'ingalls', 28210),
 (u'Steve in Halifax', 9499),
 (u'StewartSR', 8185),
 (u'kavurt', 6573),
 (u'BrianCumminger', 3851),
 (u'dgenge', 2569),
 (u'Dmajackson', 2147),
 (u'whitebuffalo', 1977)]


In [28]:
# Top 10 types of food

cur.execute('''
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='cuisine'
GROUP BY value
ORDER BY num DESC
LIMIT 10
''')
all_rows = cur.fetchall()
print "5) Top 10 types of food"
pprint(all_rows)

5) Top 10 types of food
[(u'pizza', 20),
 (u'burger', 17),
 (u'coffee_shop', 13),
 (u'chinese', 8),
 (u'greek', 6),
 (u'sandwich', 5),
 (u'sushi', 4),
 (u'japanese', 3),
 (u'mediterranean', 3),
 (u'mexican', 3)]


In [29]:
# Top 5 sport facilities

cur.execute('''
SELECT ways_tags.value, COUNT(*) as count
FROM ways_tags
WHERE key='sport'
GROUP BY ways_tags.value
ORDER BY count DESC
LIMIT 5
''')
all_rows = cur.fetchall()
print "6) Top 5 sport facilities"
pprint(all_rows)

6) Top 5 sport facilities
[(u'baseball', 88),
 (u'soccer', 57),
 (u'tennis', 39),
 (u'basketball', 36),
 (u'golf', 27)]


In [31]:
# Looks at the postal codes in the database

cur.execute('''
SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as tags 
WHERE tags.key='postcode' 
GROUP BY tags.value 
ORDER BY count DESC
LIMIT 5
''')
all_rows = cur.fetchall()
print "7) Top 5 Halifax postal codes"
pprint(all_rows)

7) Top 5 Halifax postal codes
[(u'B3T 2H8', 42),
 (u'B2W 4A2', 38),
 (u'B2W 2X6', 36),
 (u'B3S 1G2', 36),
 (u'B2W 2W5', 33)]


In [32]:
# Sort cities by count

cur.execute('''
SELECT tags.value, COUNT(*) as count
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags
WHERE tags.key='city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 5
''')
all_rows = cur.fetchall()
print "8) List of top 5 cities in Halifax Regional Municipality"
pprint(all_rows)

8) List of top 5 cities in Halifax Regional Municipality
[(u'Halifax', 7315),
 (u'Dartmouth', 4290),
 (u'Cole Harbour', 1718),
 (u'Eastern Passage', 1159),
 (u'Westphal', 720)]


In [33]:
# Number of contributions made in 2016

cur.execute('''
SELECT COUNT(*) as count
FROM (SELECT timestamp FROM nodes UNION ALL SELECT timestamp FROM ways) as i
WHERE i.timestamp LIKE '%2016%'
''')
all_rows = cur.fetchall()
print "9) Number of contributions made in 2016"
pprint(all_rows)

9) Number of contributions made in 2016
[(9330,)]


In [34]:
conn.close()

# Conclusion and Suggestions for Improvement

By bringing the Halifax OpenStreetMap data into a database and performing queries on it, we can pull out a lot of interesting information about the dataset. However, by looking at some of the results, there is a lot of room for improvement. One way that would improve this data is to remove any further inconsistencies, as was done during the audit for street name abbreviations and postal codes. This would save the data analyst a lot of time since much of their energy goes towards wrangling the data. 

OpenStreetMap could implement an audit each time a user contributes to their data, in order to ensure that similar data is written in a consistent format. For example, if a postal code in Halifax was inputed as b3h2j1, OpenStreetMap would convert this data into the standard format B3H 2J1. 

Alternatively, rather than having OpenStreetMap audit all the data that enters their system, they could encourage users to edit other people's contributions. If it were set up in a similar was as how Wikipedia double checks the credibility of each contribution made on their site, the data on OpenStreetMap would more likely be cleaner data and therefore would require less wrangling.

The benefits of improving these inconsistencies with the data is that it would clean up and ensure that everything is written in the same format. On the downside, OpenStreetMap may not want to change the input generated by its users. There is also a chance that this type of audit does not change every piece of data to the correct format, causing additional errors in the data.

Although there are still a few errors that could be fixed with the dataset, I think that the queries I performed have still given me a good idea of the Halifax OpenStreetMap layout.